### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

### GOOGLE SHEETS

In [3]:
comp = descarga('1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw','Competencias')

Descarga Correcta!


In [4]:
comp_type = {}
for i in comp['Country'].unique():
    comp_type[i] = comp[comp['Country'] == i]['Tipo'].unique()[0]

### CONSTANTES

In [5]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    fin = str(today - relativedelta(days=1))
    inicio = str((today - relativedelta(days=1) - relativedelta(months=2)).replace(day=1))
else:
    fin = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))
    inicio = str((today - relativedelta(months=2)).replace(day=1))
week = today.isocalendar()[1]
week_max = week-1
week_min = week-5
month = fin[:-3]

In [6]:
# Me traigo el rr
rr = func_rr()

### QUERIES

In [7]:
q_crm = '''WITH wallet_table AS (
    SELECT DISTINCT wa.order_id AS order_id,
           d.campaignCode AS campaign,
           d.amount AS amount
    FROM `peya-bi-tools-pro.il_wallet.fact_wallet_attributions` AS wa,
    UNNEST (attributionDetails) AS d
    WHERE wa.operation = 'Purchase'),
    wc_table AS (
    SELECT cn.country_name AS country,
           c.city_name AS city,
           a.area_name AS area,
           o.registered_date AS fecha,
           wa.campaign AS campaign,
           COUNT(DISTINCT o.order_id) AS orders,
           COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS acq,
           SUM(CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN wa.amount / ce.rate_us ELSE 0 END) AS amount,
           SUM(CASE WHEN bi.payment_mode != 'TOTAL_AMOUNT' THEN wa.amount / ce.rate_us ELSE 0 END) AS amount_shopper
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    INNER JOIN `peya-bi-tools-pro.il_core.fact_order_funding` AS w ON o.order_id = w.order_id AND w.type IN (25,35)
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
    LEFT JOIN wallet_table AS wa ON o.order_id = wa.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
    WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2,3,4,5)
SELECT wct.country AS Country,
       FORMAT_DATE('%Y-%m',wct.fecha) AS Month,
       EXTRACT(WEEK(MONDAY) FROM wct.fecha) AS Week,
       IFNULL(wct.city,'-') AS City,
       IFNULL(wct.area,'-') AS Area,
       wct.campaign AS Campaign,
       'WALLET' AS Benefit,
       SUM(wct.orders) AS Orders_CRM,
       SUM(wct.acq) AS Acq_CRM,
       SUM(wct.amount) AS Amount,
       SUM(wct.amount_shopper) AS Amount_Shopper
FROM wc_table AS wct
GROUP BY 1,2,3,4,5,6

UNION ALL

SELECT cn.country_name AS Country,
       FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       EXTRACT(WEEK(MONDAY) FROM o.registered_date) AS Week,
       IFNULL(c.city_name,'-') AS City,
       IFNULL(a.area_name,'-') AS Area,
       tca.talon_campaign_name AS Campaign,
       'TALON COUPON' AS Benefit,
       COUNT(DISTINCT o.order_id) AS Orders_CRM,
       COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS Acq_CRM,
       SUM(CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS Amount,
       SUM(CASE WHEN bi.payment_mode != 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS Amount_Shopper
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
INNER JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON tc.talon_campaign_id = tca.talon_campaign_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.order_status = 'CONFIRMED'
GROUP BY 1,2,3,4,5,6'''.format(inicio,fin)

In [8]:
# Descargo la data
bq_crm = pd.io.gbq.read_gbq(q_crm, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████| 855448/855448 [03:46<00:00, 3772.65rows/s]


In [9]:
# Copio las bases
crm = bq_crm.copy()

### TRABAJO

In [10]:
# Trabajo CRM
val = [i for i in crm.columns if i not in ['Country','Month','Week','City','Area','Campaign','Benefit']]
crm[val] = crm[val].astype(float)
col = ['Country','City','Area','Campaign','Benefit']
crm[col] = crm[col].apply(lambda x: x.astype(str).str.upper())
crm['Campaign'] = crm['Campaign'].str.replace(' ', '')

In [11]:
# Marco las campañas a filtrar
crm['Filtrar'] = crm['Campaign'].apply(filtrar_cam)
crm = crm[crm['Filtrar'] == 'No'].copy()
# Trabajo CRM
crm['Type'] = crm['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
crm['Type Wallet'] = crm.apply(lambda x: filtros(x['Campaign'],'TIPOS',x['Benefit']),axis=1)
crm['Segment'] = crm['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
crm['Campaña'] = crm['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
crm['Budget'] = crm['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
crm['Trial'] = crm['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
# Coloco el IVA
crm = func_iva(crm)
# Saco el IVA
used = 'Amount'
used_shopper = 'Amount_Shopper'
crm['Final'] = crm[used_shopper] / (1 + crm['IVA']) * crm['Extra'] + crm[used]

In [12]:
# Coloco el tipo de Competencia
crm['Competitive Type'] = crm['Country'].apply(lambda x: comp_type[x] if x in list(comp_type.keys()) else '-')
crm['Union'] = crm.apply(lambda x: str(x['Country'])+'-'+str(x['City'])+'-'+str(x['Area']) if x['Competitive Type'] == 'CITY-AREA' else str(x['Country'])+'-'+str(x['City'])+'-',axis=1)
# Hago un merge con la competencia
crm = crm.merge(comp[['Union','Competencia']],on=['Union'],how='left')
crm['Competencia'].replace([np.nan,np.inf,-np.inf],'LOW',inplace=True)
# Marco pais sin Competencia
crm['Pais Competido'] = crm['Country'].apply(lambda x: 'SI' if x in list(comp_type.keys()) else 'NO')

In [13]:
# Coloco la division entre Active/Churn/Reactive
active = ['WELCOME',"'7-15"]
churn = ['15-30']
reactive = ["'+30",'30-90','90-180','180']
crm['RMO Idea'] = '-'
crm['RMO Idea'] = crm.apply(lambda x: 'ACTIVE USERS' if x['Segment'] in active else x['RMO Idea'],axis=1)
crm['RMO Idea'] = crm.apply(lambda x: 'CHURN PREVENTION' if x['Segment'] in churn else x['RMO Idea'],axis=1)
crm['RMO Idea'] = crm.apply(lambda x: 'REACTIVATION' if x['Segment'] in reactive else x['RMO Idea'],axis=1)

In [14]:
# Armo la PT Final
index = ['Country','Pais Competido','Month','Week','Competencia','Budget','Type','Type Wallet','Segment','Campaña','Trial','RMO Idea']
values = ['Orders_CRM','Acq_CRM','Final']
final = crm.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()
# Ordeno el resultado
final.sort_values(['Country','Month','Week','Competencia','Budget','Type','Type Wallet','Segment','Campaña','RMO Idea'],inplace=True)

In [15]:
# Creo los RR
final['Orders RR'] = final.apply(lambda x: round(x['Orders_CRM'] / rr,0) if x['Month'] == month else x['Orders_CRM'],axis=1)
final['Acq RR'] = final.apply(lambda x: round(x['Acq_CRM'] / rr,0) if x['Month'] == month else x['Acq_CRM'],axis=1)
final['Final RR'] = final.apply(lambda x: round(x['Final'] / rr,0) if x['Month'] == month else x['Final'],axis=1)
final[str(today)] = final['Month'].apply(lambda x: rr if x == month else 1)
final.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [16]:
# Filtro las Budget NONE
final = final[final['Budget'] != 'NONE'].copy()

In [17]:
# Coloco las marcas de Semanas
final['Week Type'] = 'Current'
final['Week Type'] = final.apply(lambda x: 'Last 5 Complete' if week_min <= x['Week'] <= week_max else x['Week Type'],axis=1)
final['Week Type'] = final.apply(lambda x: 'Before' if x['Week'] < week_min else x['Week Type'],axis=1)

### CARGA

In [18]:
carga(final,'1Y5SR_oFtBPJzkpz1r944Iv9UoRutW7fg7kvqYGG5WJc','Crudo')

Carga Correcta!
